In [1]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence, text
# fix random seed for reproducibility
numpy.random.seed(7)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [13]:
# read and replace null values
heavywater_dataset_df = pd.read_csv("/Users/channa/Projects/heavywater/document-classification-test/shuffled-full-set-hashed.csv", 
            names = ['class', 'doc'])

heavywater_dataset_df.dropna(inplace = True)


In [11]:
# convert dataframe to matrix
# matrix values have tfidf of each word
num_words_to_keep = 2000
tokenizer_obj = text.Tokenizer(num_words = num_words_to_keep, 
                   filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', 
                   lower=True, 
                   split=' ', 
                   char_level=False, 
                   oov_token=None, 
                   document_count=0)
tokenizer_obj.fit_on_texts(heavywater_dataset_df.doc)
encoded_docs = tokenizer_obj.texts_to_matrix(heavywater_dataset_df.doc, mode='count')

In [14]:
heavywater_dataset_df.replace({'class' : { 'DELETION OF INTEREST' : 1, 
                                            'RETURNED CHECK' : 2, 
                                            'BILL' : 4, 
                                            'POLICY CHANGE' : 5, 
                                            'CANCELLATION NOTICE' : 6, 
                                            'DECLARATION' : 7, 
                                            'CHANGE ENDORSEMENT' : 8, 
                                            'NON-RENEWAL NOTICE' : 9, 
                                            'BINDER' : 10, 
                                            'REINSTATEMENT NOTICE' : 11, 
                                            'EXPIRATION NOTICE' : 12, 
                                            'INTENT TO CANCEL NOTICE' : 13, 
                                            'APPLICATION' : 14, 
                                            'BILL BINDER' : 15, }}, inplace=True)
y = np.array(heavywater_dataset_df['class'])

In [15]:
X_train, X_test, y_train, y_test = train_test_split(encoded_docs, y, test_size=0.33, random_state=42)

In [16]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((41646, 2000), (41646,), (20513, 2000), (20513,))

In [ ]:
# # truncate and pad input sequences
# max_review_length = 2000
# X_train = sequence.pad_sequences(X_train, maxlen = max_review_length, dtype = 'int32')
# X_test = sequence.pad_sequences(X_test, maxlen = max_review_length, dtype = 'int32')

In [ ]:
y_train

In [20]:
# create the model
embedding_vector_length = 32
max_review_length = 2000
model = Sequential()
model.add(Embedding(num_words_to_keep, embedding_vector_length, input_length=max_review_length))
model.add(LSTM(15))
model.add(Dense(16, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 2000, 32)          64000     
_________________________________________________________________
lstm_8 (LSTM)                (None, 15)                2880      
_________________________________________________________________
dense_8 (Dense)              (None, 16)                256       
Total params: 67,136
Trainable params: 67,136
Non-trainable params: 0
_________________________________________________________________
None
Train on 41646 samples, validate on 20513 samples
Epoch 1/3
41646/41646 [==============================] - 1575s 38ms/step - loss: 2.0097 - acc: 0.2917 - val_loss: 1.9723 - val_acc: 0.3005
Epoch 2/3
41646/41646 [==============================] - 1572s 38ms/step - loss: 1.9583 - acc: 0.3072 - val_loss: 1.9708 - val_acc: 0.3005
Epoch 3/3
41646/41646 [==============================] - 1556s 37ms/s